In [1]:
import numpy as np
import math

def gaussSeidel(iterEqs,x,tol = 1.0e-9):
    "x,numIter,omega = gaussSeidel(iterEqs,x,tol = 1.0e-9).\n Método de Gauss-Seidel para resolver Ax=b. La matriz A debe tener en su mayoría elementos nulos.\n El usuario debe proveer la función iterEqs(x,omega) que devuelve la solución mejorada.\n 'omega' es el factor de relajación."
    omega = 1.0
    k = 10
    p=1
    for i in range(1,501):
        xOld = x.copy()
        x = iterEqs(x,omega)
        dx = math.sqrt(np.dot(x-xOld,x-xOld))
        if dx < tol: 
            return x,i,omega
        #Calcula la relajación después de k+p iteraciones 
        if i == k:
            dx1 = dx
        if i == k + p:
            dx2 = dx
            omega = 2.0/(1.0 + math.sqrt(1.0- (dx2/dx1)**(1.0/p)))
    print("Gauss-Seidel falló para converger.")

In [2]:
def conjGrad(Av,x,b,tol=1.0e-9):
    "x, numIter = conjGrad(Av,x,b,tol=1.0e-9).\n Método del gradiente conjugado para resolver Ax=b.\n La matriz A en su mayoría debe contener elementos nulos.\n El usuario debe proveer la función Av(v) que devuelve el vector Av."
    n = len(b)
    r = b - Av(x)
    s = r.copy()
    for i in range(n):
        u = Av(s)
        alpha = np.dot(s,r)/np.dot(s,u)
        x = x + alpha*s
        r = b - Av(x)
        if(math.sqrt(np.dot(r,r))) < tol:
            break
        else:
            beta = -np.dot(r,u)/np.dot(s,u)
            s = r + beta*s
    return x,i

### Ejemplo 26
Escriba un programa que resuelva el siguiente sistema de ecuaciones con el método de Gauss-Seidel con relajación para cualquier valor de $n$. Ejecute el programa con $n=20$.
$$
\begin{pmatrix}
2 & -1 & 0 & 0 & \cdots & 0 & 0 & 0 & 1\\
-1 & 2 & -1 & 0 & \cdots & 0 & 0 & 0 & 0\\
0 & -1 & 2 & -1 & \cdots & 0 & 0 & 0 & 0\\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots & \vdots\\
0 & 0 & 0 & 0 & \cdots & -1 & 2 & -1 & 0\\
0 & 0 & 0 & 0 & \cdots & 0 & -1 & 2 & -1\\
1 & 0 & 0 & 0 & \cdots & 0 & 0 & -1 & 2
\end{pmatrix}
\begin{pmatrix}
x_1\\
x_2\\
x_3\\
\vdots\\
x_{n-2}\\
x_{n-1}\\
x_n
\end{pmatrix}
=\begin{pmatrix}
0\\
0\\
0\\
\vdots\\
0\\
0\\
1
\end{pmatrix}
$$
_Solución_: Note que las fórmulas iterativas en este caso son:
$$x_1=\frac{\omega}{2}(x_2-x_n)+(1-\omega)x_1$$
$$x_i=\frac{\omega}{2}(x_{i-1}-x_{i+1})+(1-\omega)x_1, \ i=2,3,...,n-1.$$
$$x_n=\frac{\omega}{2}(1-x_1-x_{n-1})+(1-\omega)x_n$$

In [3]:
def iterEqs(x,omega):
    n = len(x)
    x[0] = omega*(x[1] - x[n-1])/2.0 + (1.0 - omega)*x[0]
    for i in range(1,n-1):
        x[i] = omega*(x[i-1] + x[i+1])/2.0 + (1.0 - omega)*x[i]
        x[n-1] = omega*(1.0 - x[0] + x[n-2])/2.0 + (1.0 - omega)*x[n-1]
    return x
n = eval(input("Ingrese el número de ecuaciones: "))
x = np.zeros(n)
x,numIter,omega = gaussSeidel(iterEqs,x)
print("Número de iteraciones =",numIter)
print("Factor de relajación =",omega)
print("La solución es:\n",x)

Número de iteraciones = 256
Factor de relajación = 1.6988309117973837
La solución es:
 [-4.50000000e+00 -4.00000000e+00 -3.50000000e+00 -3.00000000e+00
 -2.50000000e+00 -2.00000000e+00 -1.50000000e+00 -9.99999998e-01
 -4.99999997e-01  2.62929058e-09  5.00000003e-01  1.00000000e+00
  1.50000000e+00  2.00000000e+00  2.50000000e+00  3.00000000e+00
  3.50000000e+00  4.00000000e+00  4.50000000e+00  5.00000000e+00]


### Ejemplo 27
Resuelva el ejemplo anterior con el método del gradiente conjugado.

_Solución:_ Para la matriz de coeficientes dada, las componentes del vector Ax(v) son:
$$(Ax)_1=2v_1-v_2+v_n$$
$$(Ax)_i=-v_{i-1}+2v_i-v_{i+1}, \ i=2,3,...,n-1.$$
$$(Ax)_n=-v_{n-1}+2v_n+v_1$$

In [4]:
def Ax(v):
    n = len(v)
    Ax = np.zeros(n)
    Ax[0] = 2.0*v[0] - v[1]+v[n-1]
    Ax[1:n-1] = -v[0:n-2] + 2.0*v[1:n-1] -v[2:n]
    Ax[n-1] = -v[n-2] + 2.0*v[n-1] + v[0]
    return Ax
n = eval(input("Ingrese el número de ecuaciones : "))
b = np.zeros(n)
b[n-1] = 1.0
x = np.zeros(n)
x,numIter = conjGrad(Ax,x,b)
print("Número de iteraciones :",numIter)
print("La solución es:\n",x)

Número de iteraciones : 9
La solución es:
 [-4.5 -4.  -3.5 -3.  -2.5 -2.  -1.5 -1.  -0.5  0.   0.5  1.   1.5  2.
  2.5  3.   3.5  4.   4.5  5. ]


### Ejercicio 27
Resolver las ecuaciones
$$
\begin{pmatrix}
4 & -1 & 1\\
-1 & 4 & -2\\
1 & -2 & 4
\end{pmatrix}
\begin{pmatrix}
x_1\\
x_2\\
x_3
\end{pmatrix}
=\begin{pmatrix}
12\\
-1\\
5
\end{pmatrix}
$$
por el método de Gauss-Seidel sin relajación.
_Solución:_ Note que las fórmulas iterativas en este caso son:
$$x_1=\frac{1}{4}(12+x_2-x_3)$$
$$x_2=\frac{1}{4}(-1+x_1+2x_3)$$
$$x_3=\frac{1}{4}(5-x_1+2x_2)$$

In [5]:
x = np.zeros(3)
def gaussSeidel(x,tol = 1.0e-9):
    for i in range(1,501):
        xOld = x.copy()
        x[0]=(12+x[1]-x[2])/4
        x[1]=(-1+x[0]+2*x[2])/4
        x[2]=(5-x[0]+2*x[1])/4
        dx = math.sqrt(np.dot(x-xOld,x-xOld))
        if dx < tol: 
            return x,i
    print("Gauss-Seidel falló para converger.")
x,numIter = gaussSeidel(x)
print("Número de iteraciones =",numIter)
print("La solución es:\n")
for i in range(len(x)):
    print("x",i+1,"=",x[i])

Número de iteraciones = 14
La solución es:

x 1 = 2.999999999981825
x 2 = 1.0000000000661957
x 3 = 1.0000000000376414


### Ejercicio 28
Resuelva el ejercicio anterior con el método del gradiente conjugado.

_Solución:_ Para la matriz de coeficientes dada, las componentes del vector Ax(v) son:
$$(Ax)_1=4v_1-v_2+v_3$$
$$(Ax)_2=-v_1+4v_2-2v_3$$
$$(Ax)_3=v_1-2v_2+4v_3$$

In [6]:
def Ax(v):
    Ax = np.zeros(3)
    Ax[0]=4*v[0]-v[1]+v[2]
    Ax[1]=-v[0]+4*v[1]-2*v[2]
    Ax[2]=v[0]-2*v[2]+4*v[2]
    return Ax
b = np.array([12,-1,5])
x = np.zeros(3)
def conjGrad(Av,x,b,tol=1.0e-9):
    r = b - Av(x)
    s = r.copy()
    for i in range(3):
        u = Av(s)
        alpha = np.dot(s,r)/np.dot(s,u)
        x = x + alpha*s
        r = b - Av(x)
        if(math.sqrt(np.dot(r,r))) < tol:
            break
        else:
            beta = -np.dot(r,u)/np.dot(s,u)
            s = r + beta*s
    return x,i
x,numIter = conjGrad(Ax,x,b)
print("Número de iteraciones :",numIter)
print("La solución es:\n")
for i in range(len(x)):
    print("x",i+1,"=",x[i])

Número de iteraciones : 2
La solución es:

x 1 = 2.9660907033165436
x 2 = 1.1651243789152521
x 3 = 1.1683773198389824
